## Steps
1. Get input from users for the topic
2. Get top Google serp results for that 
3. Scrap text
4. Clean text
5. Perform keywords extraction
6. Keep the above task running every 1 day

7. Seperate dashboard to check this online


In [9]:
#Importing the modules
import pandas as pd


In [10]:
## Step 1: user input from input.xlsx file
df=pd.read_excel("input.xlsx",sheet_name="input")
inputs = df['topics'].to_list()
url_to_fetch_count=df['urls_to_fetch'].to_list()

In [11]:
## Step 2: Google serp results for inputs making df
from googlesearch import search
import pandas as pd
l=[]
for index, inp in enumerate(inputs):
    for url in search(inp,tld='com', lang='en', num=10, start=0, stop=url_to_fetch_count[index], pause=2.0,country='India'):
        print(inp, url)
        l.append(url)

df = pd.DataFrame(l,columns=['url'])
# df

hyundai https://www.hyundai.com/in/en
hyundai https://www.hyundai.com/worldwide/en
hyundai http://www.hyundai.com/worldwide/en
hyundai https://commons.wikimedia.org/wiki/File:Hyundai_Motor_Company_logo.svg
hyundai https://en.wikipedia.org/wiki/Hyundai_Motor_Company
Tata https://www.tatamotors.com/
Tata http://www.tatamotors.com/
Tata https://www.logotaglines.com/tata-motors-logo-taglines/
Tata https://en.wikipedia.org/wiki/Tata_Motors
Tata https://twitter.com/orfonline/status/1452235926676119552?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet


In [12]:
## Step 3: Scraping text for each url
import requests
from bs4 import BeautifulSoup
def get_page(url):
    try:
        print(f'Working on {url}')
        return BeautifulSoup(requests.get(url).content,'html.parser')
    except:
        return BeautifulSoup('','html.parser')

df['page_content']=df['url'].apply(get_page)
# df

Working on https://www.hyundai.com/in/en
Working on https://www.hyundai.com/worldwide/en
Working on http://www.hyundai.com/worldwide/en
Working on https://commons.wikimedia.org/wiki/File:Hyundai_Motor_Company_logo.svg
Working on https://en.wikipedia.org/wiki/Hyundai_Motor_Company
Working on https://www.tatamotors.com/
Working on http://www.tatamotors.com/
Working on https://www.logotaglines.com/tata-motors-logo-taglines/
Working on https://en.wikipedia.org/wiki/Tata_Motors
Working on https://twitter.com/orfonline/status/1452235926676119552?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Etweet


In [13]:
# Checking a sample text
# df.iloc[4,1].text

In [14]:
## Step 4: Text cleaning
pass

In [15]:
## Step 5: Keywords extractions 

# using pke - https://boudinfl.github.io/pke/build/html/unsupervised.html#textrank
import nltk
import pke
import os
os.system('pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz')
os.system('python -m spacy link en_core_web_sm en')
nltk.download('stopwords')

# define the set of valid Part-of-Speeches
pos = {'NOUN', 'PROPN', 'ADJ'}

# 1. create a TextRank extractor.
extractor = pke.unsupervised.TextRank()

# 2. load the content of the document.
extractor.load_document(input=' '.join(df.iloc[:,1].apply(lambda x:x.text).to_list()),
                        language='en',
                        normalization=None)

# 3. build the graph representation of the document and rank the words.
#    Keyphrase candidates are composed from the 33-percent
#    highest-ranked words.
extractor.candidate_weighting(window=2,
                              pos=pos,
                              top_percent=0.33)

# 4. get the 10-highest scored candidates as keyphrases
keyphrases = extractor.get_n_best(n=20)

# keyphrases

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ghost\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
# Writing to output file

#loading to dataframe
output_df=pd.DataFrame(keyphrases,columns=['keyphrases','text_rank_score'])

# saving in the ouput excel file
output_df.to_excel("output.xlsx",sheet_name="output_keyphrases",index=False)
print("Wrote to file!!")

Wrote to file!!
